## Baseline Calculation (This notebook is not executable, instead run the experiment.ipynb files)

In [1]:
import networkx as nx
import numpy as np
from collections import defaultdict
import itertools

In [ ]:
def find_add(graph):   #this function finds and adds a new virtual entry node only if there are mult. entry nodes present 
    
    # collect nodes with no incoming edges
    roots = []
    for node, indeg in graph.in_degree():
        if indeg == 0:
           roots.append(node)

    l = len(roots) #length

    # create single entry node
    if l > 1:
        entry_node = 0
        graph.add_node(entry_node)

        # link entry node to each root
        for r in roots:
            graph.add_edge(entry_node, r, weight=DEFAULT_WEIGHT_VALUE)

        return entry_node, graph, roots
    else:
       # only one root, that's the entry node
       return roots[0], graph, roots

In [ ]:
def merge_targets(origgraph):  # this function merges the targets
    
    # identifying nodes with no outgoing edges
    targets = []
    for node, outdegree in origgraph.out_degree():
        if outdegree == 0:
            targets.append(node)

    # count how many targets
    l = len(targets)

    # if there is 0 or 1 target, no need to merge
    if l <= 1:
        return origgraph

    # convert target nodes into string labels
    tstrings = []
    for t in targets:
        tstrings.append(str(t))

    # join target strings into merged label
    merged_label = "c("
    for i in range(len(tstrings)):

        merged_label += tstrings[i]
        if i != len(tstrings) - 1:
            merged_label += ","
    merged_label += ")"

    # create a new graph
    newgraph = nx.MultiDiGraph()

    # collect non-target nodes
    non_targets = []
    for node in origgraph.nodes():
        if node not in targets:
            non_targets.append(node)

    # add all non-target nodes to new graph
    for n in non_targets:
        newgraph.add_node(n)

    # add the merged virtual target node
    newgraph.add_node(merged_label)

    # track edges that originally pointed to targets
    pred_targetedges = {}
    for u, v, data in origgraph.edges(data=True):
        if v in targets:
            if u not in pred_targetedges:
                pred_targetedges[u] = []
            # explicitly handle default weight
            if 'weight' in data:
                w = data['weight']
            else:
                w = DEFAULT_WEIGHT_VALUE
                
            pred_targetedges[u].append((w, v))

    # recreate edges to the virtual target
    for u, edges in pred_targetedges.items():
        # count number of occurrences for each weight
        wcounts = {}
        for weight, _ in edges:
            if weight not in wcounts:
                wcounts[weight] = 1
            else:
                wcounts[weight] += 1

        # add edges according to counts
        for weight, count in wcounts.items():
            for _ in range(count):
                newgraph.add_edge(u, merged_label, weight=weight)

    # copy over all other edges not involving target nodes
    for u, v, data in origgraph.edges(data=True):
        if u not in targets and v not in targets:
            edge_attr = {}
            for key in data:
                edge_attr[key] = data[key]
            newgraph.add_edge(u, v, **edge_attr)

    # return the new graph
    return newgraph

In [ ]:
# add virtual starting node to G and store other info for later use
entry, attack_graph, original_roots = find_add(attack_graph)
# merge the targets
attack_graph = merge_targets(attack_graph)

# Find targets manually since your function doesn't return them
target_list = [n for n in attack_graph.nodes() if attack_graph.out_degree(n) == 0]

In [ ]:
# Enumerate all paths(attack strats) 
# defense points
def setup_game(attack_graph, entry, target_list):
     
    target = target_list[0]
    # Get simple paths to target
    routes = list(nx.all_simple_paths(attack_graph, entry, target)) # Single target after condensation
    
    # Get nodes from all routes
    V = list(set(node for route in routes for node in route))
    
    # Node ordering from topological sort
    try:
        node_order = list(nx.topological_sort(attack_graph))
        node_order = [n for n in node_order if n in V]
    except:
        node_order = V
    
    # Defender action space 
    act = 'as1'
    if act not in globals():
        as1 = [n for n in V if n not in [entry, "attacker_entry_node"] + target_list] # Excluding trivial nodes
    
    as2 = routes  # Attacker action space
    
    # Adversary list
    adv_list = [n for n in V if n not in [entry] + target_list]
    
    return V, routes, as1, as2, adv_list, target, node_order

V, routes, as1, as2, adv_list, target, node_order = setup_game(attack_graph, entry, target_list)

In [ ]:
# Set default rates if not externally defined
if 'defense_rate_list' not in globals():
    defense_rate_list = [0]
if 'attack_rate_list' not in globals():
    attack_rate_list = [0]

starting_points = [n for n in V if n not in target_list]

In [ ]:
# Iterating all combos of defense parameters

for defense_rate in defense_rate_list:
    for attack_rate in attack_rate_list:

        # cumulative probability map
        U = {node: 0.0 for node in V}
        
        for i in as1:
            for j in starting_points:
                
                try:
                    route = nx.shortest_path(attack_graph, j, target)
                except:
                    continue
                
                pdf_d = random_steps(route, attack_rate, defense_rate, attack_graph)
                #print(route)
                try:
                    cut_point = route.index(i) + 1
                except ValueError:
                    cut_point = len(route)
                
                payoff_distr = 0

                # normalize probs up to cutoff point
                if sum(pdf_d[:cut_point]) == 0:
                    payoff_distr = [0] * cut_point
                    payoff_distr[cut_point-1] = 1
                else:
                    total = sum(pdf_d[:cut_point])
                    payoff_distr = [p/total for p in pdf_d[:cut_point]]
                
                for idx, node in enumerate(route[:cut_point]):
                    prob = payoff_distr[idx] * (1/len(as1)) * (1/len(V))
                    U[node] += prob
        
        print(f"Defense rate = {defense_rate}, Attack rate = {attack_rate}")
        print(f"Target {target} hit probability: {U[target]:.6f}\n")